### **1. Librerias**

In [1]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Access variables
email = os.getenv('email')
password = os.getenv('password')

print(f"Email: {email}")

Email: elianapinto20@outlook.com


In [2]:
# librerias 
import pandas as pd
import sys
import time


In [3]:
import warnings

# Ignorar warnings de pandas sobre cambios futuros
warnings.filterwarnings('ignore', category=FutureWarning)
import logging


In [4]:
# Librerias Selenium
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By  # buscar por etiquetas, clases, ids, paths, xpaths
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

### **2. Webscrapping data**

In [6]:
def pagina_principal(url):
    # agregar opciones al driver
    options = Options()

    # Ajustar el custom User-Agent, evitar bloqueos facilmente
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36")

    # Inicializar el webdriver con las opciones personalizadas. 
    driver = webdriver.Chrome(options=options)

    # maximizar pantalla
    driver.maximize_window()

    # Entra a la pagina
    driver.get(url)
    return driver

In [7]:
def sign_in(driver):


    try:
        # Espera explícita para el enlace de inicio de sesión
        sign_in_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="signIn"]/div/div/a'))
        )
        sign_in_link.click()

        # Espera explícita para el botón de inicio de sesión
        sign_in_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.gr-button.gr-button--dark.gr-button--auth.authPortalConnectButton.authPortalSignInButton'))
        )
        sign_in_button.click()

        # Encuentra el campo de correo electrónico y envía el correo
        email_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "ap_email"))
        )
        email_input.send_keys(f"{email}")

        # Encuentra el campo de contraseña y envía la contraseña
        password_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "ap_password"))
        )
        password_input.send_keys(f"{password}")

        # Encuentra y haz clic en el botón de envío
        sign_in_submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "signInSubmit"))
        )
        sign_in_submit_button.click()

    except Exception as e:
        print(f"Se produjo un error: {e}")


In [8]:
### NEW ! 
# def iniciar_y_logear(driver, url):
#     # Llamamos a la función para abrir la página
#     driver = iniciar_driver(url)
    
#     # Llamamos a la función para iniciar sesión después de abrir la página
#     sign_in(driver)

# # Llamamos a la función que incluye ambas acciones
# iniciar_y_logear(driver, 'http://example.com')


In [9]:
def go_to_all_genres(driver):
    
    # Espera hasta que el span con el texto "Browse ▾" sea clickeable
    browse_span = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='Browse ▾']"))
    )

    # Haz clic en el span
    browse_span.click()


    # Espera hasta que el enlace "All Genres" sea clickeable
    all_genres_link = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//a[text()='All Genres']"))
    )

    # Haz clic en el enlace "All Genres"
    all_genres_link.click()

In [10]:
def create_dict():
    books_inf = {
    'books': [],
    'writer': [],
    'rating': [],
    'description': [],
    'statistics': [],
    'details': [],
    'urls': [],
    'genres': [],
    'reading': [],
    'will_read': [],
    'isbn': [],
    'ediciones': [],
    'published': [],
    'format': [],
    'asin': [],
    'language': []
    }
    return books_inf


In [11]:
# def get_genres(driver):
#     try:
#         # Encuentra todos los nombres de géneros
#         elements = driver.find_elements(By.CSS_SELECTOR, ".bigBoxContent a.gr-hyperlink")
#         names = [element.text for element in elements]
#     except TimeoutException:
#         logging.error("Error general: No se pudo cargar la lista de géneros (Timeout).")
#     except NoSuchElementException:
#         logging.error("Error: No se encontraron los géneros en la página.")
#     except WebDriverException as e:
#         logging.error(f"Error del navegador")
#     except Exception as e:
#         logging.error(f"Error inesperado: {e}")

#     return names[5:6]

In [12]:
# //[@class="actionLink" and contains(text(), "More {gen} books...")]
        
        ## AQUI VA UNA FUNCION QUE BUSCA LOS LIBROS Y LUEGO LOS REVISA Y SACA LA INFO 
        ## DEBO VOLVER ATRAS PORQUE TIENE VARIOS LINKS LA PAGINA O? 


        # # Regresar nuevamente a la página anterior
        # driver.back()
        # time.sleep(2)  # Pausa para permitir la carga de la página

In [13]:
# def select_a_genre(driver, genres):

#     for gen in genres:
#         # Este rango parece estar limitado a 1 género, puedes ajustar el rango según sea necesario
#         logging.info(f"Procesando género: {gen}")
        
#         try:
#             # Esperar que el enlace sea clickeable y hacer clic
#             art_link = WebDriverWait(driver, 20).until(
#                 EC.element_to_be_clickable((By.XPATH, f"//a[contains(text(), '{gen}')]"))
#             )
#             art_link.click()
#             time.sleep(1)  # Pausa para permitir la carga de la página
            
#             logging.info(f"Navegando a: {driver.current_url}")
#             return gen
        

#         except TimeoutException:
#             logging.error(f"Error: No se pudo hacer clic en el enlace para '{gen}' (Timeout).")
#         except NoSuchElementException:
#             logging.error(f"Error: No se encontró el enlace para '{gen}'.")
#         except WebDriverException as e:
#             logging.error(f"Error del navegador: {e}")


In [14]:
# def get_links(driver): 
#     try: 
#         logging.info("Proceeding with scraping book URLs...")
#         # Usamos un conjunto para evitar duplicados
#         URLs = set()
        
#         # Encontramos los elementos <a> que empiezan con '/book/show'
#         anchor_tags = driver.find_elements(By.XPATH, "//a[starts-with(@href, '/book/show')]")
        
#         # Iteramos sobre los elementos encontrados
#         for tag in anchor_tags:
#             url = tag.get_attribute('href')
#             if url in URLs:
#                 logging.info(f"Duplicate URL skipped: {url}")
#             else:
#                 URLs.add(url)
#                 logging.info(f"Found unique book URL: {url}")
        
#         # Convertimos el conjunto en una lista para devolverlo
#         return list(URLs)
    
#     except Exception as e:
#         logging.error("Error occurred while trying to find book URLs", exc_info=True)
#         return []


In [15]:
# def scroll_until_found(driver, xpath):
#     for _ in range(10):  # Intenta 10 desplazamientos como máximo
#         try:
#             button = driver.find_element(By.XPATH, xpath)
#             driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
#             return button
#         except NoSuchElementException:
#             driver.execute_script("window.scrollBy(0, 500);")
#             time.sleep(1)  # Espera para permitir que se cargue más contenido
#     raise Exception(f"No se encontró el elemento con XPath: {xpath}")

# def explore_books(driver, gen): 
#     try:
#         gen_formatted = gen.lower()  # Convierte "art" a "Art"
#         xpath = f'//div[@class="moreLink"]/a[contains(text(), "More {gen_formatted} books...")]'
#         button = scroll_until_found(driver, xpath)
#         button.click()
#         url_books = get_links(driver)
#         print(f"Botón 'More {gen} books...' clickeado exitosamente.")
#     except Exception as e:
#         print(f"No se pudo hacer clic en el botón: {e}")
    
#     return url_books


In [16]:
# logging.info(f"Navegando a: {driver.current_url}")

# # Ahora buscar los enlaces en esta página
# urls = get_links()
# for url in urls:
#     get_information(url, gen)

In [17]:
url = "https://www.goodreads.com" 

In [18]:
driver = pagina_principal(url)

In [19]:
sign_in(driver)

In [20]:
# go_to_all_genres(driver)

In [21]:
dict_dataframe = create_dict()

In [22]:
dict_dataframe

{'books': [],
 'writer': [],
 'rating': [],
 'description': [],
 'statistics': [],
 'details': [],
 'urls': [],
 'genres': [],
 'reading': [],
 'will_read': [],
 'isbn': [],
 'ediciones': [],
 'published': [],
 'format': [],
 'asin': [],
 'language': []}

In [23]:
books_links = pd.read_csv('reviews_goodreads_tarea_clean.csv')['book_id']
urls = list(set(list(books_links)))
urls

[49807365,
 6,
 5,
 199458836,
 91717683,
 294965,
 10944570,
 294972,
 688191,
 193790015,
 294977,
 61243455,
 65605,
 294986,
 294988,
 8552536,
 220987481,
 60194905,
 60489817,
 28180574,
 21954673,
 124944498,
 60457077,
 43843702,
 30965883,
 34930815,
 59342980,
 58065033,
 62980235,
 50659468,
 50659467,
 50659471,
 50659472,
 58720403,
 79331476,
 214368406,
 3342492,
 29622429,
 65994917,
 50626726,
 60784812,
 24182961,
 38731956,
 6029507,
 61210821,
 54165709,
 54493401,
 51904734,
 36634846,
 17563886,
 75858168,
 6455548,
 218169599,
 25624832,
 202408199,
 43876624,
 17727768,
 51839258,
 36569376,
 70844716,
 40861997,
 25559368,
 210829641,
 79331661,
 337,
 40829270,
 343,
 38469981,
 61931873,
 229733,
 86245736,
 61407601,
 48890225,
 3932533,
 36012407,
 38732162,
 58392966,
 33784207,
 20316563,
 25756072,
 66421161,
 55083434,
 195035565,
 28213684,
 27918777,
 48497096,
 49545689,
 199262689,
 41091554,
 196706805,
 134971894,
 6193662,
 58065414,
 37683751,
 

In [24]:
def create_link_books(urls):
    urls_books_complete = []
    url_base = 'https://www.goodreads.com/book/show/'
    for i in urls:
        url_complete = f"{url_base}{i}"
        urls_books_complete.append(url_complete)
    return urls_books_complete


urls_to_scrap = create_link_books(urls)

In [25]:
missing_links = []

In [26]:
dict_dataframe

{'books': [],
 'writer': [],
 'rating': [],
 'description': [],
 'statistics': [],
 'details': [],
 'urls': [],
 'genres': [],
 'reading': [],
 'will_read': [],
 'isbn': [],
 'ediciones': [],
 'published': [],
 'format': [],
 'asin': [],
 'language': []}

In [27]:
def get_principal_info(driver):
    # Extract book name
    try:
        names_book = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.Text.Text__title1')))
        name = names_book.text.strip()
        dict_dataframe['books'].append(name if name else None)
    except (NoSuchElementException, TimeoutException) as e:
        logging.error(f"Error extracting book name from {url}: {e}")
        dict_dataframe['books'].append(None)

    # Extract authors
    try:
        autores = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'span.ContributorLink__name')))
        name_writer = [autor.text.strip() for autor in autores if autor.text.strip()]
        dict_dataframe['writer'].append(name_writer if name_writer else None)
    except (NoSuchElementException, TimeoutException) as e:
        logging.error(f"Error extracting authors from {url}: {e}")
        dict_dataframe['writer'].append(None)

    # Extract rating
    try:
        ratings_num = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.RatingStatistics__rating')))
        ratings_num_text = ratings_num.text.strip()
        dict_dataframe['rating'].append(ratings_num_text if ratings_num_text else None)
    except (NoSuchElementException, TimeoutException) as e:
        logging.error(f"Error extracting rating from {url}: {e}")
        dict_dataframe['rating'].append(None)

    # Extract description
    try:
        desc = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span.Formatted')))
        description = desc.text.strip()
        dict_dataframe['description'].append(description if description else None)
    except (NoSuchElementException, TimeoutException) as e:
        logging.error(f"Error extracting description from {url}: {e}")
        dict_dataframe['description'].append(None)

    # Extract statistics
    try:
        statics = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'RatingStatistics__meta')))
        ratings_book_views = statics[0].get_attribute('aria-label').strip() if statics else None
        dict_dataframe['statistics'].append(ratings_book_views if ratings_book_views else None)
    except (NoSuchElementException, TimeoutException) as e:
        logging.error(f"Error extracting statistics from {url}: {e}")
        dict_dataframe['statistics'].append(None)

    # Extract genres
    try:
        # option 1 
        # Locate the element containing the genres
        genres_element = driver.find_element(By.CLASS_NAME, 'CollapsableList')
        # Extract all genre texts
        genres = genres_element.find_elements(By.CLASS_NAME, 'Button__labelItem')


        # option 2
        # Wait for the genres section to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='genresList']")))
        # Locate the genre elements
        genres = driver.find_elements(By.CSS_SELECTOR, "[data-testid='genresList'] .Button__labelItem")

        # Extract and append genre names
        genre_list = [genre.text for genre in genres]
        dict_dataframe['genres'].append(genre_list if genre_list else None)

    except Exception as e:
        logging.error(f"Error extracting genres from {url}: {e}")
        dict_dataframe['genres'].append(None)

    # Extract additional details
    try:
        detail = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.FeaturedDetails p')))
        dets = [det.text.strip() for det in detail if det.text.strip()]
        dict_dataframe['details'].append(dets if dets else None)
        
    except (NoSuchElementException, TimeoutException) as e:
        logging.error(f"Error extracting details from {url}: {e}")
        dict_dataframe['details'].append(None)

In [28]:
def who_is_reading(driver):
    try:
        # Esperar a que las señales sociales estén presentes
        captions = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".SocialSignalsSection__caption"))
        )

        # Verificar si se encontraron suficientes elementos
        caption_texts = [caption.text for caption in captions]
        if len(caption_texts) > 1:
            # Añadir el texto de la señal de lectura
            dict_dataframe['reading'].append(caption_texts[0] if caption_texts[0] else None)
            # Añadir el texto de la señal de querer leer
            dict_dataframe['will_read'].append(caption_texts[1] if caption_texts[1] else None)
        elif len(caption_texts) == 1:
            # Añadir el texto de la señal de lectura
            dict_dataframe['reading'].append(caption_texts[0] if caption_texts[0] else None)
            # Añadir el texto de la señal de querer leer
            dict_dataframe['will_read'].append(None)


    except (NoSuchElementException, TimeoutException) as e:
        logging.error(f"Error al encontrar los elementos de señales sociales: {e}")
        # Si ocurre un error, agregar None en las dos categorías
        dict_dataframe['reading'].append(None)
        dict_dataframe['will_read'].append(None)


In [29]:
# def get_editions(driver):
#         # Extract additional details

#     try:
#         # Esperar a que el elemento con el texto de ediciones esté presente
#         editions_element = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, "//a[contains(@href, '/work/editions/') and contains(@class, 'Button--tertiary')]"))
#         )
        
#         # Obtener el texto del elemento
#         editions_text = editions_element.find_element(By.CLASS_NAME, "Button__labelItem").text.strip()
#         dict_dataframe['ediciones'].append(editions_text)  # Registrar el texto obtenido

#     except TimeoutException:
#         logging.warning("El elemento 'Show all' no está presente en esta URL.")
#         dict_dataframe['ediciones'].append(None)  # Registrar que no había texto

#     except NoSuchElementException as e:
#         logging.error(f"Error al buscar el texto de ediciones: {e}")
#         dict_dataframe['ediciones'].append(None)  # Registrar que no había texto

#     except Exception as e:
#         logging.warning(f"No se pudo obtener el texto de ediciones: {e}")
#         dict_dataframe['ediciones'].append(None)  # Registrar que no se pudo obtener el texto

In [30]:
def get_info_details(dirver):
     # Wait for the "Book details and editions" button and click it
    try:
        button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'Button--inline') and contains(@aria-label, 'Book details and editions')]"))
        )
        button.click()
        logging.info("Botón de detalles encontrado y clickeado.")
    except (NoSuchElementException, TimeoutException) as e:
        logging.error(f"Error finding or clicking the button: {e}")

    # Wait for the elements that contain 'Format', 'Published', 'ASIN', and 'Language'
        # Wait for the "EditionDetails" section to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "EditionDetails"))
    )

    try:
        # Esperar a que el elemento con el texto de ediciones esté presente
        editions_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(@href, '/work/editions/') and contains(@class, 'Button--tertiary')]"))
        )
        
        # Obtener el texto del elemento
        editions_text = editions_element.find_element(By.CLASS_NAME, "Button__labelItem").text.strip()
        dict_dataframe['ediciones'].append(editions_text)  # Registrar el texto obtenido

    except TimeoutException:
        logging.warning("El elemento 'Show all' no está presente en esta URL.")
        dict_dataframe['ediciones'].append(None)  # Registrar que no había texto

    except NoSuchElementException as e:
        logging.error(f"Error al buscar el texto de ediciones: {e}")
        dict_dataframe['ediciones'].append(None)  # Registrar que no había texto

    except Exception as e:
        logging.warning(f"No se pudo obtener el texto de ediciones: {e}")
        dict_dataframe['ediciones'].append(None)  # Registrar que no se pudo obtener el texto


    # Locate the rows in the details section
    rows = driver.find_elements(By.CSS_SELECTOR, ".EditionDetails .DescListItem")

    # Extract and append the label and content for each row
    for row in rows:
        label = row.find_element(By.TAG_NAME, "dt").text  # Extract the label (e.g., "Format", "Published")
        content = row.find_element(By.CSS_SELECTOR, ".TruncatedContent__text").text  # Extract the content
        
        # Append content based on the label
        if "format" in label.lower():
            dict_dataframe['format'].append(content if content else None)
            
        elif "published" in label.lower():
            dict_dataframe['published'].append(content if content else None)
            
        elif "asin" in label.lower():
            dict_dataframe['asin'].append(content if content else None)
            
        elif "isbn" in label.lower():
            dict_dataframe['isbn'].append(content if content else None)
            
        elif "language" in label.lower():
            dict_dataframe['language'].append(content if content else None)
    

In [31]:
dict_dataframe_no = [] # esta declarado fuera 
def get_information(driver, url):
    
    # dict_dataframe['urls'].append(url)

    try:
        logging.info(f"Processing URL: {url}")  # Log the URL being processed

        if url:
            # # Navigate to the book's page
            dict_dataframe['urls'].append(url)
            driver.get(url)

            get_principal_info(driver)

            who_is_reading(driver)

            # get_editions(driver)

            # get_info_details(driver)

    except Exception as e:
        logging.error(f"An unexpected error occurred for URL {url}: {e}")  
        dict_dataframe_no.append(url)

In [ ]:
for i in urls_to_scrap[500:1500]:
    
    get_information(driver, i)

ERROR:root:Error extracting genres from https://www.goodreads.com: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7632580D5+2992373]
	(No symbol) [0x00007FF762EEBFD0]
	(No symbol) [0x00007FF762D8590A]
	(No symbol) [0x00007FF762D8C46F]
	(No symbol) [0x00007FF762D8F11C]
	(No symbol) [0x00007FF762D8F1EF]
	(No symbol) [0x00007FF762DD31CB]
	(No symbol) [0x00007FF762DFF36A]
	(No symbol) [0x00007FF762DCC596]
	(No symbol) [0x00007FF762DFF580]
	(No symbol) [0x00007FF762E1F584]
	(No symbol) [0x00007FF762DFF113]
	(No symbol) [0x00007FF762DCA918]
	(No symbol) [0x00007FF762DCBA81]
	GetHandleVerifier [0x00007FF7632B6A2D+3379789]
	GetHandleVerifier [0x00007FF7632CC32D+3468109]
	GetHandleVerifier [0x00007FF7632C0043+3418211]
	GetHandleVeri

In [55]:
dict_dataframe



{'books': ['El vampiro de la colonia Roma',
  'Harry Potter and the Goblet of Fire',
  'Harry Potter and the Prisoner of Azkaban',
  'Aquí todos mienten',
  'Heaven',
  'Rurouni Kenshin, Volume 28',
  'Cuerpos descosidos',
  'Rurouni Kenshin, Volume 19',
  'Everlost',
  'El verano en que mi madre tuvo los ojos verdes',
  'Rurouni Kenshin, Volume 18',
  'Tú y otros desastres naturales',
  None,
  'Rurouni Kenshin, Volume 23',
  'Rurouni Kenshin, Volume 25',
  'El cuarto protocolo',
  'Sombra y hueso',
  'Rendirse al diablo',
  'Di que Seré Tuyo',
  'El hombre que ríe',
  'La sociedad del cansancio',
  'Unos ojos recién inaugurados',
  'Sueños robados',
  'Manual de resistencia',
  'Eva: Alfa y Omega',
  'Gone Rogue',
  'Relatos de vampiros',
  'Lessons in Chemistry',
  'Conejo maldito',
  'A Court of Mist and Fury',
  'A Court of Thorns and Roses',
  'A Court of Frost and Starlight',
  'A Court of Wings and Ruin',
  'Apenas hombre',
  'Contigo en el éter',
  'Bailar lento',
  'Por mano 

In [56]:
for i, val in dict_dataframe.items():
    print(i, len(val))

books 500
writer 500
rating 500
description 500
statistics 500
details 500
urls 500
genres 500
reading 500
will_read 500
isbn 0
ediciones 0
published 0
format 0
asin 0
language 0


In [57]:
import json
json_string = json.dumps(dict_dataframe)
print(json_string)  

{"books": ["El vampiro de la colonia Roma", "Harry Potter and the Goblet of Fire", "Harry Potter and the Prisoner of Azkaban", "Aqu\u00ed todos mienten", "Heaven", "Rurouni Kenshin, Volume 28", "Cuerpos descosidos", "Rurouni Kenshin, Volume 19", "Everlost", "El verano en que mi madre tuvo los ojos verdes", "Rurouni Kenshin, Volume 18", "T\u00fa y otros desastres naturales", null, "Rurouni Kenshin, Volume 23", "Rurouni Kenshin, Volume 25", "El cuarto protocolo", "Sombra y hueso", "Rendirse al diablo", "Di que Ser\u00e9 Tuyo", "El hombre que r\u00ede", "La sociedad del cansancio", "Unos ojos reci\u00e9n inaugurados", "Sue\u00f1os robados", "Manual de resistencia", "Eva: Alfa y Omega", "Gone Rogue", "Relatos de vampiros", "Lessons in Chemistry", "Conejo maldito", "A Court of Mist and Fury", "A Court of Thorns and Roses", "A Court of Frost and Starlight", "A Court of Wings and Ruin", "Apenas hombre", "Contigo en el \u00e9ter", "Bailar lento", "Por mano propia. Estudio sobre la pr\u00e1ctic

In [58]:
# Save the dictionary to a JSON file
with open('data.json', 'w') as json_file:
    json.dump(dict_dataframe, json_file, indent=4)  # indent for pretty printing

In [59]:
import json

# Specify the path to your JSON file
file_path = 'data.json'  # Replace with your actual file path

# Open and load the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# Print the data (or process it as needed)
print(data)


{'books': ['El vampiro de la colonia Roma', 'Harry Potter and the Goblet of Fire', 'Harry Potter and the Prisoner of Azkaban', 'Aquí todos mienten', 'Heaven', 'Rurouni Kenshin, Volume 28', 'Cuerpos descosidos', 'Rurouni Kenshin, Volume 19', 'Everlost', 'El verano en que mi madre tuvo los ojos verdes', 'Rurouni Kenshin, Volume 18', 'Tú y otros desastres naturales', None, 'Rurouni Kenshin, Volume 23', 'Rurouni Kenshin, Volume 25', 'El cuarto protocolo', 'Sombra y hueso', 'Rendirse al diablo', 'Di que Seré Tuyo', 'El hombre que ríe', 'La sociedad del cansancio', 'Unos ojos recién inaugurados', 'Sueños robados', 'Manual de resistencia', 'Eva: Alfa y Omega', 'Gone Rogue', 'Relatos de vampiros', 'Lessons in Chemistry', 'Conejo maldito', 'A Court of Mist and Fury', 'A Court of Thorns and Roses', 'A Court of Frost and Starlight', 'A Court of Wings and Ruin', 'Apenas hombre', 'Contigo en el éter', 'Bailar lento', 'Por mano propia. Estudio sobre la prácticas suicidas', 'El desorden que dejas', '

In [60]:
pop_column = ['isbn', 'ediciones', 'published', 'format', 'asin', 'language']
for column in data.copy().keys():
    if column in pop_column:
        data.pop(column)

In [61]:
books = pd.DataFrame(data)

In [62]:
books.to_csv('books.csv', index=False)

In [63]:
df = pd.read_csv('books.csv')

In [64]:
df.isna().sum()

books            7
writer           7
rating           7
description     13
statistics       7
details          8
urls             0
genres         156
reading          9
will_read       28
dtype: int64